In [1]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm.notebook import tqdm
import os
from pympler import asizeof
import plotly
import plotly.graph_objects as go
import plotly.express as px


In [2]:
# scv=pd.read_csv('ratings.csv', index_col=False)  
# keys=scv["timestamp"]
# sort_keys=keys.sort_values(axis=0).drop_duplicates()
# sort_keys.reset_index(drop= True , inplace= True)

scv=pd.read_csv("cinema.csv", index_col=False)
sort_keys=scv["timestamp"]
# import base36
data=[[key,ind] for ind,key in enumerate(sort_keys)]


In [3]:
import import_ipynb
from plr import PLR
print(len(PLR(data, 100, show=False)))


4
18 [830792475, 1]


2
2
1143


In [4]:
class GreedySpline:
    def __init__(self, error=10):
        self.error=error
        self.greedyspline=[]

    def start(self,new_point):
        self.tg_err_top=float("inf")
        self.tg_err_bot=float("-inf")
        self.greedyspline.append(new_point)
        self.greedyspline.append(new_point)
        self.last_point=new_point
    
    def tg_spline(self, a,b,err=0):
        return (a[1]+self.error*err-b[1])/(a[0]-b[0])
        
    def new_coridor (self,a,b):
        self.tg_err_top=min(self.tg_err_top, self.tg_spline(a,b,1))
        self.tg_err_bot=max(self.tg_err_bot, self.tg_spline(a,b,-1))
        
    def new_spline(self, new_point): 
        if not self.greedyspline:
            self.start(new_point)
            return
        tg_new_point=self.tg_spline(new_point,self.greedyspline[-2])
        if tg_new_point<self.tg_err_top and tg_new_point>self.tg_err_bot:
            self.new_coridor(new_point, self.greedyspline[-2])
            self.greedyspline[-1]=new_point
        else:
            self.greedyspline.append(self.last_point)
            self.tg_err_top=self.tg_spline(new_point, self.greedyspline[-1],1)
            self.tg_err_bot=self.tg_spline(new_point, self.greedyspline[-1],-1)
        self.last_point=new_point 
        
        
        
    def load_data(self,data):
        for i in data:
            self.new_spline(i)
    
    def plot(self,fig):
        dataX=[]
        dataY=[]
        for i,j in  self.greedyspline:
            dataX.append(i)
            dataY.append(j)
        fig.add_trace(go.Scatter(x=dataX,y=dataY, name="G",  mode='lines+markers',
                marker=dict(
                    color="green",
                    size=10,
                ),))
        return fig
    
        
        
    
class RadixSpline:
    def __init__(self,GreedySpline,bitcount):
        self.bitcount=bitcount
        self.greedyspline=GreedySpline.greedyspline
        self.error=GreedySpline.error
        self.radixspline=[None]*2**self.bitcount
        self.bit_len=self.bit_lenght(self.greedyspline[-1][0])
        # print("bit_len",self.bit_len)
        self.len_greedy=len(self.greedyspline)
        self.not_prefics,self.mod=self.not_prefics_count()
        self.move_right=self.bit_len-self.bitcount-self.not_prefics
        # print("not_prefics",self.not_prefics)
        # print("mod", self.mod)
        # print("move_right",self.move_right)
        self.get_RadixSpline()
        
    def bit_lenght(self,n):
        return(len(bin(n).lstrip('-0b')))
    def not_prefics_count(self):
        n=self.greedyspline[0][0]
        count,pref=0,0
        
        if self.bit_lenght(n)==self.bit_len:
            count=self.bit_len-self.bit_lenght(((1<<self.bit_len)-1)^n)
        for i in range(count):
            pref=(pref<<1)+1
        pref<<=(self.bit_len-count)
        return count,pref
        
        
    
    ##Способ построение радикс сплайна, нужно заменить
    def first_bits(self,n):
        if self.not_prefics:
           n%=self.mod
        #    print(n)
        return n>>self.move_right
    
    def get_RadixSpline(self):
        
        for ind in range(len(self.greedyspline)):
            temp=self.first_bits(self.greedyspline[ind][0])
            # print(temp)
            if not self.radixspline[temp]:
                self.radixspline[temp]=ind
        if  self.radixspline[2**self.bitcount-1]==None:
             self.radixspline[2**self.bitcount-1]=len(self.greedyspline)-1
        for i in range (2**self.bitcount-1,-1,-1):
            if self.radixspline[i]==None:                
                self.radixspline[i]=self.radixspline[i+1]
        return self.radixspline
    
    def bin_search_spline(self,left,right,key):
        while left<=right:
            mid=(left+right)//2
            if key>= self.greedyspline[mid][0] :
                if key<=self.greedyspline[mid+1][0] :
                   return [mid,mid+1]
                else: 
                   left=mid+1
            else:
                right=mid-1


    
        
    def give_answer(self,key):
        first_bit=self.first_bits(key)
        left=self.radixspline[first_bit]-1
        if first_bit==len(self.radixspline)-1:
            right=len(self.greedyspline)-1
        else:
            right=self.radixspline[first_bit+1]
        # left=0
        # right=len(self.greedyspline)-1
        bin_ans=self.bin_search_spline(left,right,key)
        kl,pl=self.greedyspline[bin_ans[0]]
        kr,pr=self.greedyspline[bin_ans[1]]

        return pl+(key-kl)*(pr-pl)//max(1,(kr-kl))
    
    def plot(self, fig):
        
        dataX=[]
        dataY=[]
        for i in  self.radixspline:
            dataX.append(self.greedyspline[i][0])
            dataY.append(self.greedyspline[i][1])
        # plt.plot(dataX,dataY, marker='o', markersize=15)
        fig.add_trace(go.Scatter(x=dataX,y=dataY, name="R",
                                  mode='lines+markers',
                marker=dict(
                    color="red",
                    size=20,
                ),))
        return fig

In [5]:
def plot_data(data,fig,error=0 ):
    

    dataX=[]
    errT=[]
    errB=[]
    dataY=[]
    for i,j in  data:
        dataX.append(i)
        dataY.append(j)
        if error:
            errB.append(j-error)
            errT.append(j+error)
    fig.add_trace(go.Scatter(x=dataX,y=dataY, name="data",
                             mode="markers",
                marker=dict(
                    color="blue",
                    size=5,
                ),))
    
    # print(data.__sizeof__())
    if error:
        fig.add_trace(go.Scatter(x=dataX,y=errB, name="-err"))
        fig.add_trace(go.Scatter(x=dataX,y=errT, name="+err"))     
    return fig   

def get_spline(data, ERR,bitcount ,plot=[False,False]): 
    G=GreedySpline(error=ERR)
    G.load_data(data)
    
    # print(len(G.greedyspline)/len(data)) 
    R=RadixSpline(G, bitcount)
    if plot[0]:
        fig=go.Figure()
        
        fig=G.plot(fig)
        fig=R.plot(fig)
        fig=plot_data(data,fig,plot[1]*ERR )
        fig.show()
    return (G,R)

In [6]:
A=get_spline(data[:10000],5,6,[True,True])

In [7]:
# scv=pd.read_csv('amazon_com.csv', dtype={"ASIN":bytes}, index_col=False)  
# keys=scv["ASIN"]
# sort_keys=keys.sort_values(axis=0).drop_duplicates()
# sort_keys.reset_index(drop= True , inplace= True)
# # sort_keys.to_csv("amazon.csv")
# sort_keys[1]
# import base36
# data=[[base36.loads(key),ind] for ind,key in enumerate(sort_keys)]
# data=data

In [8]:
def bin_search(data, key, left, right):
    while left <= right:
        mid = (left + right) // 2  # Находим середину массив
        if data[mid][0] == key:  # Если целевое значение найдено
            return mid
        elif data[mid][0] < key:  # Если целевое значение справа
            left = mid + 1
        else:  # Если целевое значение слева
            right = mid - 1
    return -1  

    
def get_key(data, spline, key ):
    radix_answer=spline[1].give_answer(key[0])
    left=radix_answer-spline[0].error
    right=radix_answer+spline[0].error
    res=bin_search(data, key[0], max(left,data[0][1]),min(right, data[-1][1]))
    # print(res)
    if res!=key[1]: raise Exception("FAIL", res, key, math.floor(left),math.ceil(right))
    return res
  
def check_bin_search(data, key):
    res=bin_search(data, key[0],data[0][1], data[-1][1])
    if res!=key[1]: print ("FAIL binsearch")
    return res

## Тестирование сплайна

создание одного

Прогон, поиск всех ключей, для проверки приблизительного времени 

Сравнение с приблизительным временем бинпоиска

In [9]:
for i in tqdm(data):
    check_bin_search(data,i)

  0%|          | 0/630112 [00:00<?, ?it/s]

Перебор ошибки, для проверки времени

In [11]:
bitcount = 8
if 0:
    for i in range(50,1000,50):
        print("i=", i)
        start1=time.time()
        A=get_spline(data,i,bitcount)
        end1 =time.time()
        start2=time.time()
        for j in data:
            get_key(data,A,j)
        end2=time.time()
        print("gen",(end1-start1)//0.001,"|search",(end2-start2)//0.001,"|greedy_size",A[1].len_greedy, "size(bytes)= ", asizeof.asizeof(A) )
        
        print("_"*14)
    

"Эталонное" время бинпоиска

In [12]:
start2=time.time()
for i in data:    
    check_bin_search(data,i)
end2=time.time()
print((end2-start2)//0.001)

1302.0


Создание Фенс поинтов

In [13]:
scv=pd.read_csv('rating.csv', index_col=False)  
keys=scv["timestamp"]
sort_keys=keys.sort_values(axis=0).drop_duplicates()
sort_keys.reset_index(drop= True , inplace= True)

In [14]:
case_size=4096
len_points=os.path.getsize("rating.csv")//case_size
average_size=os.path.getsize("rating.csv")//len(scv)
points=[]
print(average_size)
for i in range (0,len(sort_keys),case_size//average_size ):
    points.append(sort_keys[i])

32


In [15]:
print(len(sort_keys))
print(len(points))
case_size//average_size

630112
4923


128

Отчет по используемой памяти

In [16]:
print(os.path.getsize("rating.csv"),"       size ratings")
print(os.path.getsize("cinema.csv"),"       size cinema (обрезанный ratings)")

print( asizeof.asizeof(get_spline(data,16,20)), "  =  asize 16,20 ")
print( asizeof.asizeof(get_spline(data,2**11,8)),"  =  asize 2**11,8 ")
print( asizeof.asizeof(get_spline(data,4,4)),"  =  asize 4,4")
print( asizeof.asizeof(get_spline(data,2**20,4)),"  =  asize 2**20,4")
if 0: print( asizeof.asizeof(get_spline(data,2**20,26)),"  =  asize 2**20,26")
print(asizeof.asizeof(points), "     =asize points")

24760980        size ratings
11786810        size cinema (обрезанный ratings)
10947704   =  asize 16,20 
10656   =  asize 2**11,8 
6698600   =  asize 4,4
2224   =  asize 2**20,4
199432      =asize points


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [18]:

# fig, axs = plt.subplots(5, 5, figsize=(12, 7))
if 0:
    repeat=5
    fig = go.Figure()
    # plt.semilogx()
    err_=[2**i for i in range (2,19,2)]
    for bitcount in tqdm(range (4,27,3)):

        # time_min=2000
        # err_min=0
        x=[]
        y=[]
        for err in err_:

            # print("i=", i, end=",")
            start1=time.time()
            A=get_spline(data,err,bitcount)
            end1 =time.time()
            start2=time.time()
            for _ in range(repeat):
                for j in data:
                    get_key(data,A,j)
            end2=time.time()
            search_time=(end2-start2)/repeat//0.001
            # print((end2-start2)//0.001,"|greedy_size",A[1].len_greedy)
            y.append(search_time)
            # if time_min>search_time:
            #     time_min=search_time
            #     err_min=err
            # x.append(A[1].len_greedy)
            x.append(err)
            # print("_"*14)

        fig.add_trace(
            go.Scatter(
                x=x, y=y,  # данные
                name="bit="+str(bitcount)))
    fig.update_xaxes(type="log")
    fig.update_layout(
            xaxis=dict(
                title=dict(
                    text='spline error'
                )
            ),
            yaxis=dict(
                title=dict(
                    text='search_time'
                )
            ),
    )
    fig.show()

In [19]:
import random
x=4
import math
print("Данных всего",10**5)
random.seed(42)
for x in range(2,6):
    data_rand=[[10**4,0]]
    for i in range(1,10**5+1):
        for j in range(x):
            data_rand.append([data_rand[-1][0]+1,data_rand[-1][1]+1])
        # for j in range(x):
        data_rand.append([data_rand[-1][0]+10**5,data_rand[-1][1]+1])
    A=get_spline(data_rand,2    ,4)
    print("x=",x)
    print("Количество спалйн точек",A[1].len_greedy)
    print("Их отношение",len(data_rand)/A[1].len_greedy)


Данных всего 100000
x= 2
Количество спалйн точек 2
Их отношение 150000.5
x= 3
Количество спалйн точек 100002
Их отношение 3.999930001399972
x= 4
Количество спалйн точек 100002
Их отношение 4.999910001799964
x= 5
Количество спалйн точек 200001
Их отношение 2.99999000005


In [20]:
if 0:
        fig = go.Figure()
        x=[]
        y=[]
        for i in data_rand[:40]:
                x.append(i[0])
                y.append(i[1])
        fig.add_trace(
                go.Scatter(x=x,y=y))
        fig.show()

In [21]:
print("Данных всего",len(data_rand))
A=get_spline(data_rand,2    ,4)
print("Количество спалйн точек",A[1].len_greedy)
print("Их отношение",len(data_rand)/A[1].len_greedy)

Данных всего 600001
Количество спалйн точек 200001
Их отношение 2.99999000005


## Количество точек сплайна

In [22]:
if 1:
    this_data=data
    fig = go.Figure()
    # plt.semilogx()
    err_pow=[i/10 for i in range (10,200)]
    err_=[2**i for i in err_pow]
    bitcount=4
    diff=[]
    x=[]
    y=[]
    y_plr=[]
    for err in tqdm(err_):
        err=round(err)
        if err not in x:
        # print(err)
            A=get_spline(this_data,err,bitcount)
            y_plr.append(len(PLR(this_data, err, show=False)))
            y.append(A[1].len_greedy)
            diff.append(y[-1]/y_plr[-1])
            x.append(err)
            if A[1].len_greedy==2:
                break

    fig.add_trace(
        go.Scatter(
            x=x, y=y,  # данные
            name="Greedy"))
    fig.add_trace(
        go.Scatter(
            x=x, y=y_plr,  # данные
            name="PLR"))
    
    
    # print(time_min, "при", err_min)
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    fig.update_layout(
            # title=dict(
            #     # text='Затраченные время и память'
            # ),
            xaxis=dict(
                title=dict(
                    text='spline error'
                )
            ),
            yaxis=dict(
                title=dict(
                    text='spline point count'
                )
            ),
    )
    print(sum(diff)/len(diff))
    fig.show()


  0%|          | 0/190 [00:00<?, ?it/s]

2.199231330601321


#### время и количество точек сплайна

In [23]:
if 0:
    repeat=1
    erlystop=100
    fig = go.Figure()
    # plt.semilogx()
    err_=[int(2**i) for i in range (1,25)]
    err_1=[]
    for i in range(len(err_)-1):
        err_1.append(err_[i])
        err_1.append(err_[i]+err_[i]//2)
    for bitcount in tqdm(range (8,23,2)):
        x=[]
        y=[]
        for err in tqdm(err_1):
            start1=time.time()
            A=get_spline(data,err,bitcount)
            end1 =time.time()
            start2=time.time()
            for _ in range(repeat):
                for j in data:
                    get_key(data,A,j)
            end2=time.time()
            search_time=(end2-start2)/repeat//0.001
            # print((end2-start2)//0.001,"|greedy_size",A[1].len_greedy)
            y.append(search_time)
            # if time_min>search_time:
            #     time_min=search_time
            #     err_min=err
            x.append(A[1].len_greedy)
            if A[1].len_greedy<=erlystop: 
                break
            # x.append(asizeof.asizeof(A))
            # print("_"*14)

        fig.add_trace(
            go.Scatter(
                x=x, y=y,  # данные
                name="bit="+str(bitcount)))
        
    start2=time.time()
    for _ in range(repeat):
        for i in data:    
            check_bin_search(data,i)
        end2=time.time()
    search_time=(end2-start2)/repeat//0.001
    # fig.add_trace(
    #         go.Scatter(
    #             x=[asizeof.asizeof(points),asizeof.asizeof(points)], y=[1000,2200],  # данные
    #             name="BinSearch"))
    fig.add_trace(
            go.Scatter(
                x=[erlystop,100000], y=[search_time,search_time],  # данные
                name="BinSearch"))
        # print(time_min, "при", err_min)
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    fig.update_layout(
            title=dict(
                text='Затраченные время и память'
            ),
            xaxis=dict(
                title=dict(
                    text='spline point count'
                )
            ),
            yaxis=dict(
                title=dict(
                    text='search time'
                )
            ),
    )

    fig.show()

In [24]:
if 0:
    repeat=3
    erlystop=100
    fig = go.Figure()
    # plt.semilogx()
    err_=[int(2**i) for i in range (1,25)]
    err_1=[]
    for i in range(len(err_)-1):
        err_1.append(err_[i])
        err_1.append(err_[i]+err_[i]//2)
    err_2=[i for i in range(60,8000,100)]
    
    
    
    
    err_now=err_2
    for bitcount in tqdm(range (10,15)):
        x=[]
        y=[]
        for err in tqdm(err_now):
            start1=time.time()
            A=get_spline(data,err,bitcount)
            end1 =time.time()
            start2=time.time()
            for _ in range(repeat):
                for j in data:
                    get_key(data,A,j)
            end2=time.time()
            search_time=(end2-start2)/repeat//0.001
            # print((end2-start2)//0.001,"|greedy_size",A[1].len_greedy)
            y.append(search_time)
            # if time_min>search_time:
            #     time_min=search_time
            #     err_min=err
            # x.append(A[1].len_greedy)
            
            x.append(asizeof.asizeof(A))
            if A[1].len_greedy<=erlystop: 
                break
            # print("_"*14)

        fig.add_trace(
            go.Scatter(
                x=x, y=y,  # данные
                name="bit="+str(bitcount)))
        
    start2=time.time()
    for _ in range(repeat):
        for i in data:    
            check_bin_search(data,i)
        end2=time.time()
    search_time=(end2-start2)/repeat//0.001
    fig.add_trace(
            go.Scatter(
                x=[asizeof.asizeof(points),asizeof.asizeof(points)], y=[1300,1550],  # данные
                name="BinSearch"))
    fig.add_trace(
            go.Scatter(
                x=[20000,10**6], y=[search_time,search_time],  # данные
                name="BinSearch"))
        # print(time_min, "при", err_min)
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    fig.update_layout(
            title=dict(
                text='Затраченные время и память'
            ),
            xaxis=dict(
                title=dict(
                    text='memory used'
                )
            ),
            yaxis=dict(
                title=dict(
                    text='search time'
                )
            ),
    )

    fig.show()

In [25]:
import timeit
def all_search():
    for j in data:
                get_key(data,A,j)

In [26]:
import time
import statistics

num_runs = 20
execution_times = []
for _ in tqdm(range(num_runs)):
 start_time = time.perf_counter()
 all_search()
 end_time = time.perf_counter()
 execution_times.append(end_time - start_time)
#  print(execution_times[-1])

average_time = statistics.mean(execution_times)
stdev_time = statistics.stdev(execution_times)
print(f"Среднее время выполнения: {average_time:.6f} секунд")
print(f"Стандартное отклонение: {stdev_time:.6f} секунд")


  0%|          | 0/20 [00:00<?, ?it/s]

Среднее время выполнения: 1.762383 секунд
Стандартное отклонение: 0.022360 секунд
